In [1]:
from surprise import Dataset
import pandas as pd


# Загрузка данных
data = Dataset.load_builtin('ml-100k')

# Преобразование данных в формат pandas DataFrame
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])

# Вывод первых нескольких строк DataFrame
print(df)


      user_id item_id  rating  timestamp
0         196     242     3.0  881250949
1         186     302     3.0  891717742
2          22     377     1.0  878887116
3         244      51     2.0  880606923
4         166     346     1.0  886397596
...       ...     ...     ...        ...
99995     880     476     3.0  880175444
99996     716     204     5.0  879795543
99997     276    1090     1.0  874795795
99998      13     225     2.0  882399156
99999      12     203     3.0  879959583

[100000 rows x 4 columns]


In [2]:
# Импорт необходимых библиотек и классов
import pandas as pd  # Добавлен импорт pandas
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

svd_model = SVD()
svd_model.fit(trainset)

items = {
    'item_id': [1, 2, 3],
    'description': ['Description 1', 'Description 2', 'Description 3']
}

items_df = pd.DataFrame(items)

In [4]:
# Преобразование описаний в векторы TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(items_df['description'])

# Расчет схожести между товарами на основе TF-IDF
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [5]:
# Функция для получения рекомендаций на основе гибридной модели
def hybrid_recommendation(user_id, item_id):
    # Получение предсказания от коллаборативной модели
    collab_prediction = svd_model.predict(user_id, item_id).est

    # Получение индекса товара в контентной матрице
    item_index = items_df[items_df['item_id'] == item_id].index[0]

    # Получение схожих товаров и их схожести с текущим товаром
    sim_scores = list(enumerate(cosine_sim[item_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Получение индексов топ-3 похожих товаров
    top_items = [i[0] for i in sim_scores[1:4]]

    # Получение TF-IDF векторов описаний топ-3 похожих товаров
    top_items_tfidf = tfidf_matrix[top_items]

    # Получение среднего TF-IDF вектора для топ-3 похожих товаров
    avg_tfidf_vector = top_items_tfidf.mean(axis=0)

    # Усреднение предсказаний от обоих моделей
    hybrid_prediction = (collab_prediction + avg_tfidf_vector.sum()) / 2

    return hybrid_prediction

In [6]:
# Пример использования функции
user_id = 1
item_id = 2
recommendation = hybrid_recommendation(user_id, item_id)
print(f"Hybrid Recommendation for User {user_id}, Item {item_id}: {recommendation}")


Hybrid Recommendation for User 1, Item 2: 2.2651874999999997
